In [30]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 8)

In [24]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [25]:
df = pd.read_excel(path, sheet_name="出貨明細")

In [ ]:
product_chart = df[(df["Group"] == "華新科技")]

In [ ]:
product_chart = product_chart.groupby(["品名", "客戶料號"])[["數量"]].sum().reset_index()

In [ ]:
inpaq = product_chart["品名"].tolist()
walshin_pn = product_chart["客戶料號"].tolist()

In [ ]:
product_dict = dict(zip(walshin_pn, inpaq))

In [ ]:
robin = pd.read_clipboard()

In [ ]:
robin["INPAQ PN"] = robin["華科料號"].map(product_dict)

In [ ]:
robin.to_excel(r"C:\Users\kaihsu\Desktop\INPAQ未交.xlsx", index=False)

In [ ]:
walshin = df[(df["Group"] == "華新科技") & (df["狀態"].str.contains("已出")) & (df["BG"] == "EMC")]

In [ ]:
walshin = walshin.groupby([pd.Grouper(key="預交日期", freq="M"), "品名"])[["數量"]].sum().reset_index()

In [ ]:
walshin["Type"] = walshin["品名"].str[0:3]
walshin["Type"].unique()

In [ ]:
walshin["預交日期"] = walshin["預交日期"].dt.strftime("%Y-%m")

In [ ]:
date = walshin["預交日期"].str.split("-", expand=True)
date.columns = ["Year", "Month"]

In [ ]:
walshin = pd.concat([walshin, date], axis=1)

In [ ]:
walshin = walshin.groupby(["Year", "Month", "Type"])[["數量"]].sum().reset_index()

In [ ]:
import plotly.express as px

In [ ]:
walshin_plot = px.bar(walshin, x="Month", y="數量", color="Type", barmode="group", facet_col="Year", labels={"數量":"出貨量", "Month":"月份", "Year":"年份"})
walshin_plot.update_layout(font_family="Calibri", font_size=14, font_color="green")

In [ ]:
sorted_df = df[(df["Group"] == "華新科技") & (df["預交日期"].between("2021-03-01", "2021-12-31")) & (df["狀態"].str.contains("未出"))]

In [ ]:
sorted_df.columns

In [ ]:
result = sorted_df.groupby(["BG", "銷售單號", "銷售項次", "開單日期", "預交日期", "品名"])[["數量"]].sum().reset_index()

In [ ]:
result["交期"] = round((result["預交日期"] - result["開單日期"]).dt.days / 7, 1)

In [ ]:
result.sort_values(["數量", "交期"], ascending=False)

In [ ]:
leadtime_over = result[(result["交期"] >= 8) & (result["BG"] == "EMC")]

In [ ]:
leadtime_over["產品系列"] = leadtime_over["品名"].str[0:3]

In [ ]:
leadtime_over["產品系列"] = leadtime_over["產品系列"].map(lambda x: "Bead" if x == "MHC" or x == "MCB" else x)

In [ ]:
import plotly.express as px

In [ ]:
bar_plot = leadtime_over.groupby(["產品系列", "品名"])[["數量"]].sum().reset_index()

In [ ]:
export_plot = px.bar(bar_plot, x="產品系列", y="數量", color="品名", title="Robin哥，說好的貨呢？")
export_plot.update_layout(font_family="Calibri", font_size=15, title_font_size=26, title_font_color="red")
export_plot.show()

In [ ]:
leadtime_over = leadtime_over.sort_values("交期", ascending=False)

In [ ]:
style_df = leadtime_over.style.format({"數量":"{:,.0f}", "交期":"{:.1f}週", "開單日期":"{:%Y/%m/%d}", "預交日期":"{:%Y/%m/%d}"}).hide_index().highlight_max("數量", color="red").set_caption("交期大於8週的產品數量表(已按照交期排序)").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'red'),
        ('font-size', '20px')
    ]
}])

In [ ]:
style_df

In [ ]:
with open("EMC交期過長.html", "a") as f:
    f.write(style_df.render())
    f.write(export_plot.to_html(full_html=False, include_plotlyjs="cdn"))
    f.close()

In [12]:
'''合併Anna和趙勇的回覆'''
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()

In [13]:
for l in list(paths):
    print(l[-9:-5])

anna
yong


In [14]:
df_list = []
for l in list(paths):
    globals()["df" + l[-9:-5]] = pd.read_excel(l)
    df_list.append(globals()["df" + l[-9:-5]])

In [15]:
anna = df_list[0]
yong = df_list[1]

In [16]:
df = anna.merge(yong, on=["Order date", "下單日", "華科料號", "INPAQ PN", "PO", "ITEM", "PO+Item", "數量", "廠商ETA", "客需"], how="left")

In [18]:
df["生管回覆"] = df.apply(lambda x: x["可Pullin"] if pd.isnull(x["可Pullin"]) is not True else x["无锡生管回复"] if pd.isnull(x["无锡生管回复"]) is not True else "維持原交期", axis=1)

In [19]:
df

,Order date,下單日,華科料號,INPAQ PN,PO,ITEM,PO+Item,數量,廠商ETA,客需,可Pullin,无锡生管回复,生管回覆
0,2021.2.8,24,WLBD1005K1U601TB,MCB1005S601FBP,4502119536,210,4502119536210,170,2021.03.04,2021.03.05,NaN,3/5已出货,3/5已出货
1,2020.12.2,92,WLBD1608K1U221TB,MCB1608S221HBP,4502093073,160,4502093073160,212,2021.03.04,2021.01.05,NaN,3/4已出,3/4已出
2,2020.12.28,66,WLBD1608K1U221TB,MCB1608S221HBP,4502102726,110,4502102726110,84,2021.03.25,2021.03.05,NaN,80*3/12\n4k*3/25,80*3/12\n4k*3/25
3,2021.1.14,49,WLBD1608K1U221TB,MCB1608S221HBP,4502109767,20,450210976720,40,2021.03.04,2021.02.05,NaN,3/4已出,3/4已出
4,2021.2.18,14,WLBD1608K1U301TB,MCB1608S301HBP,4502121373,190,4502121373190,16,2021.05.27,2021.02.16,NaN,8*3/12\n8*5/27,8*3/12\n8*5/27
5,2021.2.22,10,WLCM0603Z0S2N0TB,MCI0603HQ2N0SHBP,4502122406,40,450212240640,75,2021.04.29,2021.03.06,2021-03-25 00:00:00,NaN,2021-03-25 00:00:00
6,2021.2.3,29,WLCM0603Z0S2N2TB,MCI0603HQ2N2SHBP,4502117975,210,4502117975210,15,2021.04.22,2021.02.20,2021-03-25 00:00:00,NaN,2021-03-25 00:00:00
7,2021.1.28,35,WLCM0603Z0S3N9TB,MCI0603HQ3N9SHBP,4502115602,50,450211560250,105,2021.04.22,2021.02.12,NaN,NaN,維持原交期
8,2020.12.25,69,WLCM0603Z0S6N2TB,MCI0603HQ6N2SHBP,4502102372,70,450210237270,165,2021.03.04,2021.01.06,這筆已經出完,NaN,這筆已經出完
9,2021.1.28,35,WLCM0603Z0S6N2TB,MCI0603HQ6N2SHBP,4502115602,70,450211560270,510,2021.04.22,2021.02.12,3/25可先出150K,NaN,3/25可先出150K


In [20]:
df = df[['Order date', '下單日', '華科料號', 'INPAQ PN', 'PO', 'ITEM', 'PO+Item', '數量',
       '廠商ETA', '客需', '生管回覆']]

In [21]:
df.to_excel("20210305_未交訂單pull-in生管更新.xlsx", index=False)

In [28]:
bu2 = df[(df["Group"] == "TELTONIKA") & (df["狀態"].str.contains("出") & (df["預交年份"] == 2021) & (df["預交月份"] == "March")) & (df["品名"].str.contains("RFDP"))]
bu2.to_excel(r"C:\Users\kaihsu\Desktop\bu2.xlsx")

In [31]:
pd.read_csv(r"C:\Users\kaihsu\Desktop\2001-12salesprofitBigdata_Q1.csv")

C:\Users\kaihsu\Anaconda3\envs\INPAQ_Python_project\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,5,7,8,10,12,13,19,20,22,23,24,25,28,29,30,31,33,34,35,42,44,49,50) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,年月,出貨單號,備貨單號,備貨日期,出貨日期,銷售單號,客戶名稱,業務代號,負責業務,組,產品分類,銷售類別,客戶單號,幣別,料號,品名,單位,金額(含稅),數量,單價,金額,匯率,銷售額NTD,客戶料號,出貨地,產品第三階,客戶編號,訂單業務,訂單部門,INVOICE_NO,SHIPTO,NAME,RATE,本 幣 (各廠幣別不同),Term,起運地,廠,類,BU,單位成本 (NTD),單位售價,單位毛利,毛利率%,總成本,毛利,產品前7碼,Net,RMO,RMO Q'ty,發票號碼,Rebate,客戶品名,產業-200828/0922/1123,一級,一級名,二級,二級名,部級,部級名,品牌,案名,RF/元件,"客戶料號,佳邦品名",原ERP訂單業務,Single Source,正確業務,BU.1,業一Q2四新,CS,Wistron,毛調毛利
0,2001,SGN2010060,SRF2010148,2020/1/1,2020/1/1,SON19C0553,博實結HZ,P2193,劉霞,西安行銷,MCB-1608-Sseries成,內銷,3.30E+14,CNY,300122817,MCB1608S601IBP,KPCS,1680.0,168.0,8.849558,"1,486.73",1.0000,"6,517",1111-00293,HUIZHOUCHINA(惠州),EMI成品,B1167,劉霞,西安天線,NaN,B1167,博實結HZ,4.3832,"1,486.73",DAP,SUZHOUCHINA(蘇州),S,MCB1,EMC,1000,1000,1000,-7.79%,1000,-508,NaN,"6,517",NaN,NaN,1940732417,NaN,博實結HZMCB1608S601IBP,NaN,曾明燦 總經理(兼),行銷事業處_中國行銷部,張良 處長,華東西安行銷,周雲山 經理,西安行銷組,NaN,NaN,元件,1111-00293;MCB1608S601IBP,NaN,NaN,NaN,EMC,0,華東CS,NaN,NaN
1,2001,SUG2010001,SUN2010003,2020/1/1,2020/1/1,SUB19C0540,仁寶資訊,T0038,張皓雲,高頻業二,GSM成品,外銷,4.50E+29,USD,402903100,WA-C-00-00-001,PCS,5160.0,4000.0,1.29,"5,160.00",6.9762,"157,783",DC330029D60,KUNSHANCHINA(昆山),HFM成品,CZ242,張皓雲,國內業二,HHU2010003,CZ242,仁寶資訊,4.3832,"35,997.19",DAP,SUZHOUCHINA(蘇州),S,WA-C,RF,1000,1000,1000,41.39%,1000,"65,308",NaN,"157,783",NaN,NaN,NaN,NaN,仁寶資訊WA-C-00-00-001,NB/PC,許仁傑 副總經理,行銷事業處_高頻行銷部,蔡明仁 經理,高頻行銷部,黎欣然 課長,業二,HP,MIRAMAR(EPM20),RF,DC330029D60;WA-C-00-00-001,張皓雲,DC330029D60,NaN,RF,0,Cindy,NaN,NaN
2,2001,SUG2010001,SUN2010003,2020/1/1,2020/1/1,SUB19C0540,仁寶資訊,T0038,張皓雲,高頻業二,GSM成品,外銷,4.50E+29,USD,402903209,WA-C-00-00-002,PCS,5160.0,4000.0,1.29,"5,160.00",6.9762,"157,783",DC330029D70,KUNSHANCHINA(昆山),HFM成品,CZ242,張皓雲,國內業二,HHU2010003,CZ242,仁寶資訊,4.3832,"35,997.19",DAP,SUZHOUCHINA(蘇州),S,WA-C,RF,1000,1000,1000,42.01%,1000,"66,290",NaN,"157,783",NaN,NaN,NaN,NaN,仁寶資訊WA-C-00-00-002,NB/PC,許仁傑 副總經理,行銷事業處_高頻行銷部,蔡明仁 經理,高頻行銷部,黎欣然 課長,業二,HP,MIRAMAR(EPM20),RF,DC330029D70;WA-C-00-00-002,張皓雲,DC330029D70,NaN,RF,0,Cindy,NaN,NaN
3,2001,SUG2010004,SUN2010002,2020/1/1,2020/1/1,SUB19C0521,達豐重慶,T0065,劉雅築,高頻業二,GSM成品,外銷,1075357783,USD,403502626,WA-P-LBLB-02-042,SET,670.0,1000.0,0.67,670,6.9762,"20,487",DQ6LB020404,CHONGQINGCHINA(重慶,HFM成品,CZ240,方喬毅,國內業二,HHU2010001,CZ240,達豐重慶,4.3832,"4,674.05",DAP,SUZHOUCHINA(蘇州),S,WA-P,RF,1000,1000,1000,46.62%,1000,"9,551",NaN,"20,487",NaN,NaN,NaN,NaN,達豐重慶WA-P-LBLB-02-042,NB/PC,許仁傑 副總經理,行銷事業處_高頻行銷部,蔡明仁 經理,高頻行銷部,黎欣然 課長,業二,HP,X8D,RF,DQ6LB020404;WA-P-LBLB-02-042,方喬毅,DQ6LB020404,NaN,RF,0,Kelsey,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16576,2003,RMO2030001,NaN,NaN,43895,NaN,愛傑坤,NaN,NaN,CN華東行銷,NaN,RMO,NaN,NaN,11050104,WIP201610P-R47ML,KPCS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W0031,陳廣茹,NaN,NaN,NaN,NaN,4.2660,-2131.8,NaN,NaN,W,WIP2,WIP,1000,1000,1000,NaN,1000,NaN,NaN,"-9,094","-9,094",-33.0,NaN,NaN,愛傑坤WIP201610P-R47ML,11050104,曾明燦 總經理(兼),行銷事業處_中國行銷部,張良 處長,華東西安行銷,陳廣茹 經理,華東元件組,NaN,NaN,元件,;WIP201610P-R47ML,NaN,NaN,NaN,WIP,0,華東CS,NaN,NaN
16577,2003,RMO2030001,NaN,NaN,43895,NaN,愛傑坤,NaN,NaN,CN華東行銷,NaN,RMO,NaN,NaN,11050104,WIP201610P-R47ML,KPCS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W0031,陳廣茹,NaN,NaN,NaN,NaN,4.2660,-3488.4,NaN,NaN,W,WIP2,WIP,1000,1000,1000,NaN,1000,NaN,NaN,"-14,882","-14,882",-54.0,NaN,NaN,愛傑坤WIP201610P-R47ML,11050104,曾明燦 總經理(兼),行銷事業處_中國行銷部,張良 處長,華東西安行銷,陳廣茹 經理,華東元件組,NaN,NaN,元件,;WIP201610P-R47ML,NaN,NaN,NaN,WIP,0,華東CS,NaN,NaN
16578,2003,RMO2030001,NaN,NaN,43895,NaN,愛傑坤,NaN,NaN,CN華東行銷,NaN,RMO,NaN,NaN,11050104,WIP201610P-R47ML,KPCS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W0031,陳廣茹,NaN,NaN,NaN,NaN,4.2660,-3294.6,NaN,NaN,W,WIP2,WIP,1000,1000,1000,NaN,1000,NaN,NaN,"-14,055","-14,055",-51.0,NaN,NaN,愛傑坤WIP201610P-R47ML,11050104,曾明燦 總經理(兼),行銷事業處_中國行銷部,張良 處長,華東西安行銷,陳廣茹 經理,華東元件組,NaN,NaN,元件,;WIP201610P-R47ML,NaN,NaN,NaN,WIP,0,華東CS,NaN,NaN
16579,2003,RMO2030001,NaN,NaN,43895,NaN,愛傑坤,NaN,NaN,CN華東行銷,Na